In [2]:
cursor.execute("SELECT PARAMETRO, VALOR FROM params_des WHERE ENTIDAD = 'MYSQL' AND AMBIENTE = 1")
mysql_params = {row["PARAMETRO"]: row["VALOR"] for row in cursor.fetchall()}

print("✅ Parámetros de conexión a MariaDB desde params_des:")
for k, v in mysql_params.items():
    print(f"{k} = {v}")

cursor.close()
bootstrap_conn.close()


✅ Parámetros de conexión a MariaDB desde params_des:
mysql.host = mariadb
mysql.user = root
mysql.password = StrongPass123
mysql.database = etl_config
mysql.port = 3306


In [3]:
# Usar los parámetros obtenidos para reconectar a MariaDB
conn = mysql.connector.connect(
    host=mysql_params["mysql.host"],
    user=mysql_params["mysql.user"],
    password=mysql_params["mysql.password"],
    database=mysql_params["mysql.database"],
    port=int(mysql_params["mysql.port"])
)

print("✅ Conexión exitosa a MariaDB usando parámetros de params_des")

conn.close()


✅ Conexión exitosa a MariaDB usando parámetros de params_des


In [4]:
# ===============================================
# 1. Importar librerías
# ===============================================
import mysql.connector
from pyspark.sql import SparkSession

# ===============================================
# 2. Conexión a MariaDB (bootstrap mínimo)
# ===============================================
conn = mysql.connector.connect(
    host="mariadb",       # contenedor MariaDB en tu red
    user="root",
    password="StrongPass123",
    database="etl_config"
)
cursor = conn.cursor(dictionary=True)

# ===============================================
# 3. Traer parámetros de Spark desde params_des
# ===============================================
cursor.execute("SELECT PARAMETRO, VALOR FROM params_des WHERE ENTIDAD = 'SPARK' AND AMBIENTE = 1")
spark_params = {row["PARAMETRO"]: row["VALOR"] for row in cursor.fetchall()}

print("✅ Parámetros Spark desde MariaDB:")
for k, v in spark_params.items():
    print(f"{k} = {v}")

cursor.close()
conn.close()

#


✅ Parámetros Spark desde MariaDB:
spark.master = spark://spark-master:7077
spark.executor.instances = 1
spark.executor.cores = 1
spark.executor.memory = 2g
spark.driver.memory = 1g
spark.jars = /opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar
spark.driver.extraClassPath = /opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar
spark.executor.extraClassPath = /opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar


In [5]:


# ===============================================
# 5. Iniciar SparkSession con parámetros de MariaDB
# ===============================================
spark_builder = SparkSession.builder.appName("Notebook-Spark-Con-Parametros")

spark_builder = spark_builder.master(spark_params["spark.master"])
spark_builder = spark_builder.config("spark.executor.instances", spark_params["spark.executor.instances"])
spark_builder = spark_builder.config("spark.executor.cores", spark_params["spark.executor.cores"])
spark_builder = spark_builder.config("spark.executor.memory", spark_params["spark.executor.memory"])
spark_builder = spark_builder.config("spark.driver.memory", spark_params["spark.driver.memory"])
spark_builder = spark_builder.config("spark.jars", spark_params["spark.jars"])
spark_builder = spark_builder.config("spark.driver.extraClassPath", spark_params["spark.driver.extraClassPath"])
spark_builder = spark_builder.config("spark.executor.extraClassPath", spark_params["spark.executor.extraClassPath"])

# Activar soporte Hive
spark = spark_builder.enableHiveSupport().getOrCreate()

# ===============================================
# 6. Validación
# ===============================================
print("✅ Spark version:", spark.version)
print("✅ Master URL:", spark.sparkContext.master)
print("✅ Spark levantado con parámetros desde MariaDB")

25/09/19 21:42:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark version: 2.4.5
✅ Master URL: spark://spark-master:7077
✅ Spark levantado con parámetros desde MariaDB
